# --------------------------------------------------------------------------------------------------------------
# PLAYAS de ESPAÑA 2017 : CASSANDRA
# --------------------------------------------------------------------------------------------------------------

## Inicializaciones previas

### a) Carga de librerías

In [1]:
import pandas as pd

# A) Carga de datos desde Excel a un DataFrame de PANDAS

Leemos los datos desde el fichero Excel y los desnormalizamos : leemos las 2 hojas del fichero Excel en un **mismo** DataFrame de Pandas para guardarlo todo junto.

In [2]:
df_excel_provincias = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PROVINCIAS")
df_excel_playas     = pd.read_excel("./data/playas_2017.xlsx", sheetname= "HOJA_PLAYAS")

In [3]:
df_cassandra = pd.merge(df_excel_provincias, df_excel_playas, on = ['id_provincia'], how = 'inner')
df_cassandra.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 0 to 3510
Data columns (total 12 columns):
id_provincia       3511 non-null int64
nom_provincia      3511 non-null object
comunidad          3511 non-null object
cod_playa          3511 non-null int64
nom_playa          3511 non-null object
localidad          3511 non-null object
clave_nom_playa    3511 non-null object
longitud           3511 non-null int64
longitud_txt       3511 non-null object
arena              3511 non-null object
nudista            3511 non-null object
bandera_azul       3511 non-null object
dtypes: int64(3), object(9)
memory usage: 356.6+ KB


# B) BD de CASSANDRA

## B.1) Conexión a la BD de CASSANDRA

### B.1.1) Carga de CQL

In [4]:
%load_ext cql

### B.1.2) Borrado de la BD ("playas_2017")

In [5]:
%%cql
DROP KEYSPACE playas_2017;

'No results.'

### B.1.3) Creación de la BD ("playas_2017")

In [6]:
%%cql
CREATE KEYSPACE playas_2017
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

### B.1.4) Conexión a la BD ("playas_2017")

In [7]:
%cql USE playas_2017;

'No results.'

## B.2) Creamos las TABLAS de CASSANDRA

### B.2.1) Modelo de BD de CASSANDRA

Para almacenar la información se van a crear 2 tablas :

 + Tabla de **playas**             :  contiene las playas de España y sus características en formato desnormalizado. La PK será el cod_playa que viene en el dataset de partida.
 + Tabla de **acum_longitud_prov** :  contiene la longitd de playas acumulada por provincia.

También tenemos una tabla auxiliar, creada gracias a la funcionalidad que nos da la base de datos de VISTAS MATERIALIZADAS, que vamos a utilizar para facilitar otros patrones distintos de acceso a la información.

<br><br> 

<img src="images/Modelo_BD___CASSANDRA.JPG",width=700,height=600>


<br><br>

### B.2.2) Borrado de las tablas

No aplica, ya que en pasos anteriores la BD se ha **borrado** (DROP KEYSPACE playas_2017;) y **creado** (CREATE KEYSPACE playas_2017 ...).

### B.2.3) Creación de las tablas

#### B.2.3.1) TABLA "acum_longitud_prov"

In [8]:
%%cql 

CREATE TABLE acum_longitud_prov (
    acu_nombre_prov     text,
    acu_longitud_prov   counter,
    PRIMARY KEY (acu_nombre_prov)
)

'No results.'

#### B.2.3.2) TABLA "playas"

** NOTA : ** Como señalamos anteriormente, en la tabla de **playas**, la PK será el cod_playa que viene en el dataset de partida. En caso de que no hubiese PK, se podría utilizar como PK un tipo UUID (marca temporal única) donde se guarda un TIMESTAMP con la hora de creación del registro.

In [9]:
%%cql 


CREATE TABLE playas (
    id_provincia    int,
    nom_provincia   text,
    comunidad       text,
    cod_playa       int,
    nom_playa       text,
    localidad       text,
    clave_nom_playa text,
    longitud        int,
    longitud_txt    text,
    arena           text,
    nudista         text,
    bandera_azul    text,
    PRIMARY KEY ((cod_playa), longitud, comunidad, nom_provincia, nom_playa, localidad, clave_nom_playa)
)
WITH CLUSTERING ORDER BY (longitud DESC);

'No results.'

#### B.2.3.3) Vistas materializadas

En la vista materializada no es necesario insertar datos ya que se ocupa la propia base de datos.

In [10]:
%%cql

CREATE MATERIALIZED VIEW vm_playas_by_provincia AS
   SELECT * FROM playas
   WHERE                       -- Primary key column is required to be filtered by 'IS NOT NULL'
         nom_provincia   is not NULL
     and longitud        is not NULL
     and cod_playa       is not NULL
     and nom_playa       is not NULL
     and localidad       is not NULL
     and clave_nom_playa is not NULL
     and comunidad       is not NULL
   PRIMARY KEY (nom_provincia, longitud, cod_playa, nom_playa, localidad, clave_nom_playa, comunidad)
   WITH CLUSTERING ORDER BY (longitud desc)

'No results.'

In [11]:
%%cql

CREATE MATERIALIZED VIEW vm_playas_by_comunidad AS
   SELECT * FROM playas
   WHERE                       -- Primary key column is required to be filtered by 'IS NOT NULL'
         nom_provincia   is not NULL
     and longitud        is not NULL
     and cod_playa       is not NULL
     and nom_playa       is not NULL
     and localidad       is not NULL
     and clave_nom_playa is not NULL
     and comunidad       is not NULL
   PRIMARY KEY (comunidad, longitud, nom_provincia, cod_playa, nom_playa, localidad, clave_nom_playa)
   WITH CLUSTERING ORDER BY (longitud desc)

'No results.'

# C) Carga de datos en Cassandra

In [12]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster_C = Cluster()
session_C = cluster_C.connect('playas_2017')

**NOTA :**
- **Fechas :** El **tipo** de las fechas hay que dejarlo con el tipo **Date** de Python para no tener problemas (el tipo de la columna es también Date).
- **NULL :** Los nulos en Pandas son del tipo **NumPy.nan**, por lo hay convertirlos a **None** para que la inserción en Cassandra sea correcta. Ej :

                 df_cassandra.ix[df_cassandra.nudista.isnull(), 'nudista'] = None

Definimos una **función** para insertar las playas del DataFrame en la TABLA de playas.

In [13]:
import dateutil

def fun_insertar_playas (df):
    
    sql_insert = """
INSERT INTO playas (
    id_provincia,
    nom_provincia,
    comunidad,
    cod_playa,
    nom_playa,
    localidad,
    clave_nom_playa,
    longitud,
    longitud_txt,
    arena,
    nudista,
    bandera_azul
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
    

    for index in df.index:
        
        data = [
            df.ix[index, "id_provincia"],
            df.ix[index, "nom_provincia"],
            df.ix[index, "comunidad"],
            df.ix[index, "cod_playa"],
            df.ix[index, "nom_playa"],
            df.ix[index, "localidad"],
            df.ix[index, "clave_nom_playa"],
            df.ix[index, "longitud"],
            df.ix[index, "longitud_txt"],
            df.ix[index, "arena"],
            df.ix[index, "nudista"],
            df.ix[index, "bandera_azul"]
        ]
      
        session_C.execute(sql_insert, data)
        
        session_C.execute ("UPDATE acum_longitud_prov SET acu_longitud_prov = acu_longitud_prov + %s WHERE acu_nombre_prov = %s", 
                           [df.ix[index, "longitud"], 
                            df.ix[index, "nom_provincia"]
                           ]
                          )

Ya podemos insertar los datos ...

In [14]:
fun_insertar_playas (df_cassandra)

## Consulta de datos

**Función** de utilidad que realiza un query en Cassandra y devuelve un DataFrame de Pandas

In [15]:
def fun_ejecutar_query_cassandra(sql):
    rows = session_C.execute(sql)
    return pd.DataFrame(list(rows))

# ------------------------------------------------------------------------------
# QUERYS   (Cassandra)
# ------------------------------------------------------------------------------

### QUERY_1) Las 10 playas más largas (incluída la provincia)

Esta consulta se pueder resolver así :
   1. **Primer paso :** seleccionar en la vista materializa **"vm_playas_by_provincia"**.
   2. **Segundo paso :** con ayuda de un DataFrame **ordenar** el resultado del paso anterior por "longitud".

In [16]:
sql = """
SELECT nom_provincia, localidad, nom_playa, clave_nom_playa, longitud, longitud_txt
FROM vm_playas_by_provincia
"""

df_vista_prov = fun_ejecutar_query_cassandra (sql)

In [17]:
df_vista_prov.sort_values('longitud', ascending = False).head(10)

,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
3483,Huelva,Almonte,Doñana,Almonte (Doñana),28000,28.000 metros
3484,Huelva,Almonte,Castilla,Almonte (Castilla),17000,17.000 metros
1355,Las Palmas,Pájara,El Cofete,Pájara (El Cofete),13700,13.700 metros
3485,Huelva,Lepe,Nueva Umbría,Lepe (Nueva Umbría),12000,12.000 metros
3486,Huelva,Palos de la Frontera,Mazagón,Palos de la Frontera (Mazagón),9000,9.000 metros
286,Tarragona,Deltebre,Punta del Fangar,Deltebre (Punta del Fangar),6500,6.500 metros
2368,Almería,El Ejido,Punta Entinas Sabinar,El Ejido (Punta Entinas Sabinar),6500,6.500 metros
2065,Girona,Sant Pere Pescador,Sant Pere Pescador,Sant Pere Pescador (Sant Pere Pescador),6400,6400 metros
287,Tarragona,Sant Carles de la Ràpita,El Trabucador,Sant Carles de la Ràpita (El Trabucador),6165,6.165 metros
2865,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros


### QUERY_2) Las 10 playas más largas de una cierta provincia (incluído el nombre de la provincia)

Esta consulta se pueder resolver mediante la vista materializa **"vm_playas_by_provincia"**.

In [18]:
sql = """
SELECT nom_provincia, localidad, nom_playa, clave_nom_playa, longitud, longitud_txt
FROM vm_playas_by_provincia
WHERE nom_provincia = 'A Coruña'
limit 10
"""


fun_ejecutar_query_cassandra (sql)   #  No ordenamos, porque la VISTA MATERIALIZADA ya está ordenada por longitud

,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
0,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros
1,A Coruña,Carballo,Baldaio,Carballo (Baldaio),4000,4.000 metros
2,A Coruña,Valdoviño,A Frouxeira,Valdoviño (A Frouxeira),3000,3.000 metros
3,A Coruña,Ribeira,Vilar,Ribeira (Vilar),2900,2.900 metros
4,A Coruña,Ortigueira,Morouzos,Ortigueira (Morouzos),2800,2.800 metros
5,A Coruña,Laxe,Traba,Laxe (Traba),2650,2.650 metros
6,A Coruña,Boiro,Barraña,Boiro (Barraña),2150,2.150 metros
7,A Coruña,Porto do Son,Areas Longas,Porto do Son (Areas Longas),2100,2.100 metros
8,A Coruña,Boiro,Carragueiros,Boiro (Carragueiros),2000,2.000 metros
9,A Coruña,Fisterra,Langosteira,Fisterra (Langosteira),1970,1.970 metros


### QUERY_3) Las playas de una comunidad autónoma concreta (ordenadas por longitud)

Esta consulta se pueder resolver mediante la vista materializa **"vm_playas_by_comunidad"**.

In [19]:
sql = """
SELECT comunidad, nom_provincia, localidad, nom_playa, clave_nom_playa, longitud, longitud_txt
FROM vm_playas_by_comunidad
WHERE comunidad = 'Galicia'
limit 10
"""

fun_ejecutar_query_cassandra (sql)  #  No ordenamos, porque la VISTA MATERIALIZADA ya está ordenada por longitud

,comunidad,nom_provincia,localidad,nom_playa,clave_nom_playa,longitud,longitud_txt
0,Galicia,A Coruña,Carnota,Carnota,Carnota (Carnota),5600,5.600 metros
1,Galicia,A Coruña,Carballo,Baldaio,Carballo (Baldaio),4000,4.000 metros
2,Galicia,A Coruña,Valdoviño,A Frouxeira,Valdoviño (A Frouxeira),3000,3.000 metros
3,Galicia,A Coruña,Ribeira,Vilar,Ribeira (Vilar),2900,2.900 metros
4,Galicia,A Coruña,Ortigueira,Morouzos,Ortigueira (Morouzos),2800,2.800 metros
5,Galicia,A Coruña,Laxe,Traba,Laxe (Traba),2650,2.650 metros
6,Galicia,Pontevedra,O Grove,A Lanzada,O Grove (A Lanzada),2400,2.400 metros
7,Galicia,Pontevedra,Redondela,Da Punta,Redondela (Da Punta),2400,2.400 metros
8,Galicia,A Coruña,Boiro,Barraña,Boiro (Barraña),2150,2.150 metros
9,Galicia,A Coruña,Porto do Son,Areas Longas,Porto do Son (Areas Longas),2100,2.100 metros


### QUERY_4) Las 10 provincias que mas metros de playa tienen

Esta consulta se pueder resolver así :
   1. **Primer paso :** seleccionar en el acumulado **"acum_longitud_prov"**.
   2. **Segundo paso :** con ayuda de un DataFrame **ordenar** el resultado del paso anterior por "longitud".

In [20]:
sql = """
select *
from acum_longitud_prov
"""

df_acu_prov = fun_ejecutar_query_cassandra (sql)

df_acu_prov = df_acu_prov.sort_values('acu_longitud_prov', ascending = False).head(10)

df_acu_prov.rename(columns = {"acu_nombre_prov"   : "provincia",
                              "acu_longitud_prov" : "metros_de_playa"})

,provincia,metros_de_playa
20,A Coruña,157040
16,Cádiz,138473
11,Las Palmas,137337
12,Málaga,136720
17,Almería,120765
8,Alicante/Alacant,118148
4,Illes Balears,117767
23,Huelva,113720
10,Murcia,102431
2,Tarragona,99015


### QUERY_5) Metros de playa de una cierta Comunidad Autónoma agrupados por provincia

Esta consulta se pueder resolver así :
   1. **Primer paso :** seleccionar en la vista materializa **"vm_playas_by_comunidad"**.
   2. **Segundo paso :** con ayuda de un DataFrame **agrupar por provincias** el resultado del paso anterior.

In [21]:
sql = """
SELECT comunidad, nom_provincia, nom_playa, localidad, clave_nom_playa, longitud
FROM vm_playas_by_comunidad
WHERE comunidad = 'Galicia'
"""

df_vista_comu = fun_ejecutar_query_cassandra (sql)

In [22]:
df = df_vista_comu.groupby(["comunidad", "nom_provincia"]) [["longitud"]].sum()
df = df.sort_values('longitud', ascending = False)
df = df.rename(columns = {"longitud" : "metros_de_playa"})
df

metros_de_playa
comunidad nom_provincia                 
Galicia   A Coruña                157040
          Pontevedra               93552
          Lugo                     27217

### QUERY_6) ¿ Qué Comunidad tiene más metros de playa ?

Esta consulta se pueder resolver así :
   1. **Primer paso :** seleccionar en la vista materializa **"vm_playas_by_comunidad"**.
   2. **Segundo paso :** con ayuda de un DataFrame **agrupar por comunidad** el resultado del paso anterior, hacer la suma de la **"longitud"** del cada grupo y **ordenar**.

In [23]:
sql = """
SELECT comunidad, nom_provincia, nom_playa, localidad, clave_nom_playa, longitud
FROM vm_playas_by_comunidad
"""

df_vista_comu = fun_ejecutar_query_cassandra (sql)


In [24]:
df = df_vista_comu.groupby(["comunidad"]) [["longitud"]].sum()
df = df.sort_values('longitud', ascending = False)
df.rename(columns = {"longitud" : "metros_de_playa"})

,metros_de_playa
comunidad,
Andalucía,545459
Galicia,277809
Comunitat Valenciana,276872
Cataluña/Catalunya,237274
Canarias,209374
Illes Balears,117767
"Murcia, Región de",102431
"Asturias, Principado de",70428
Cantabria,49880


### QUERY_7) ¿ Qué provincia tiene más playas ... ?

#### QUERY_7.1) ¿ Qué provincia tiene más playas ... en número ?

In [25]:
sql = """
SELECT nom_provincia, cod_playa
FROM vm_playas_by_provincia
"""

df_vista_prov = fun_ejecutar_query_cassandra (sql)   #  No ordenamos, porque la VISTA MATERIALIZADA ya está ordenada por longitud

In [26]:
res = df_vista_prov.groupby(["nom_provincia"]).cod_playa.nunique()
type(res)

pandas.core.series.Series

In [27]:
res.sort_values(ascending=False).head(5)

nom_provincia
A Coruña                  410
Pontevedra                370
Illes Balears             346
Las Palmas                322
Santa Cruz de Tenerife    257
Name: cod_playa, dtype: int64

In [28]:
df=res.to_frame()
((df.rename(columns={"cod_playa":"num_playas"})
  ).sort_values('num_playas', ascending = False)
).head(10)

,num_playas
nom_provincia,
A Coruña,410
Pontevedra,370
Illes Balears,346
Las Palmas,322
Santa Cruz de Tenerife,257
Asturias,205
Murcia,199
Girona,183
Alicante/Alacant,173


#### QUERY_7.2) ¿ Qué provincia tiene más playas ... con nombres diferentes ?

In [29]:
sql = """
SELECT nom_provincia, nom_playa
FROM vm_playas_by_provincia
"""

df_vista_prov = fun_ejecutar_query_cassandra (sql)   #  No ordenamos, porque la VISTA MATERIALIZADA ya está ordenada por longitud

In [30]:
res = df_vista_prov.groupby(["nom_provincia"]).nom_playa.nunique()
type(res)

pandas.core.series.Series

In [31]:
res.sort_values(ascending=False).head(5)

nom_provincia
A Coruña                  388
Pontevedra                350
Illes Balears             330
Las Palmas                303
Santa Cruz de Tenerife    228
Name: nom_playa, dtype: int64

In [32]:
df=res.to_frame()
((df.rename(columns={"nom_playa":"nombres_de_playas_distintos"})
  ).sort_values('nombres_de_playas_distintos', ascending = False)
).head(10)

,nombres_de_playas_distintos
nom_provincia,
A Coruña,388
Pontevedra,350
Illes Balears,330
Las Palmas,303
Santa Cruz de Tenerife,228
Asturias,199
Murcia,193
Girona,179
Alicante/Alacant,165


#### QUERY_7.3)  ¿ Qué provincia tiene más playas ... nudistas ?

In [33]:
sql = """
SELECT nom_provincia, nudista
FROM vm_playas_by_provincia
"""

df_vista_prov = fun_ejecutar_query_cassandra (sql)   #  No ordenamos, porque la VISTA MATERIALIZADA ya está ordenada por longitud

In [34]:
df_playas_nudistas = df_vista_prov [ df_vista_prov["nudista"] == u"Sí" ]   # Seleccionamos del DataFrame las 
                                                                           # playas nudistas
    
df_playas_nudistas.head(3)

,nom_provincia,nudista
22,Asturias,Sí
32,Asturias,Sí
101,Asturias,Sí


In [35]:
res = df_playas_nudistas.groupby(["nom_provincia"]).size().rename("num_playas_nudistas_puras")
type(res)

pandas.core.series.Series

In [36]:
pd.DataFrame(res).sort_values('num_playas_nudistas_puras', ascending = False).head(10)

,num_playas_nudistas_puras
nom_provincia,
Las Palmas,146
Illes Balears,79
Santa Cruz de Tenerife,27
Alicante/Alacant,18
A Coruña,17
Gipuzkoa,16
Girona,16
Pontevedra,15
Tarragona,9


#### QUERY_7.4)  ¿ Qué provincia tiene más playas ... con bandera azul ?

In [37]:
sql = """
SELECT nom_provincia, bandera_azul
FROM vm_playas_by_provincia
"""

df_vista_prov = fun_ejecutar_query_cassandra (sql)   #  No ordenamos, porque la VISTA MATERIALIZADA ya está ordenada por longitud

In [38]:
df_playas_bandera_azul = df_vista_prov [ df_vista_prov["bandera_azul"] == u"Sí" ]   # Seleccionamos del DataFrame las 
                                                                           # playas con bandera azul
    
df_playas_bandera_azul.head(3)

,nom_provincia,bandera_azul
2,Asturias,Sí
3,Asturias,Sí
12,Asturias,Sí


In [39]:
res = df_playas_bandera_azul.groupby(["nom_provincia"]).size().rename("num_playas_bandera_azul")
type(res)

pandas.core.series.Series

In [40]:
pd.DataFrame(res).sort_values('num_playas_bandera_azul', ascending = False).head(10)

,num_playas_bandera_azul
nom_provincia,
Alicante/Alacant,62
Pontevedra,57
A Coruña,45
Tarragona,43
Illes Balears,41
Murcia,40
Castellón/Castelló,33
Girona,31
Valencia/València,30


# FIN